In [104]:
import pandas as pd
import numpy as np
import re

#function that deletes the object in the index of the array and creates a new array with a copy of that object
#this is essentially append for numpy arrays
def poprow(my_array,pr):
    """ row popping in numpy arrays
    Input: my_array - NumPy array, pr: row index to pop out
    Output: [new_array,popped_row] """
    i = pr
    pop = my_array[i]
    pop = np.array(pop)
    pop = np.array(pop).astype(int)
    
    #print (pop)
    new_array = np.delete(my_array,i)
    return [new_array,pop]

#function which deletes the top values in the array to a certain number, but saves the deleted ones in a separate array using the poprow function
#this is to delete the HAZ hardness indents that landed in the FZ, but we're gonna append the FZ indents to the FZ array later ;)
def del_FZ(hardness_array, not_HAZ_number):
    """input: the hardness array and the number of points we're deleting
    output: [the new hardness array, the deleted points or FZ array]"""
    FZ_array = []
    FZ_array = np.asarray (FZ_array, dtype=int)
    
    #not_HAZ_number = int(not_HAZ_number)
    #FZ_array = np.empty(shape=not_HAZ_number, dtype=int)
    #print (FZ_array)
    hardness_array_cut = hardness_array
    x=0
    for x in range(not_HAZ_number):
        [hardness_array_cut, FZ_pop] = poprow(hardness_array_cut, 0)
        FZ_array = np.append(FZ_array, FZ_pop)
        #print (FZ_array)
    #print (FZ_array)
    return [hardness_array_cut, FZ_array]

#function which cleans up the data from the sheet, renames the columns, and returns the hardness array 
def trim_fat (hard_sheet):
    """Input: the excel spreadsheet from pandas, and the number of points that landed in the FZ (int)
    Output: the hardness in a numpy array, and the FZ hardness points also in a numpy array"""
    hard_sheet_length = len(hard_sheet.index)
    hard_sheet_new = hard_sheet.drop([0,1,2,(hard_sheet_length-1),(hard_sheet_length-2),(hard_sheet_length-3)])
    hard_sheet_new = hard_sheet_new.rename(
        columns={
            "":"index",
            "Name":"Mean Length",
            "Number of Included Replicates":"Hardness",
        })
    
    #print (hard_sheet_new)
    hardness_array =  hard_sheet_new.Hardness.to_numpy()
    #[hardness_array, FZ_array] = del_FZ(hardness_array, not_HAZ)
    return hardness_array

#this function takes the excel file name, opens the file, trims the data, removes the FZ from the HAZ and appends it to the FZ array
def get_hard_740H (filename, not_HAZ_number):
    """
    xlsx = pd.ExcelFile(filename)
    hardness_sheets = []
    for sheet in xlsx.sheet_names:
        hardness_sheets.append(xlsx.parse(sheet))
    """
    
    HAZ= pd.read_excel(filename, sheet_name=0)
    FZ= pd.read_excel(filename, sheet_name=1)
    
    HAZ_trim = trim_fat(HAZ)
    [HAZ_array, FZ_in_HAZ] = del_FZ(HAZ_trim, not_HAZ_number)
    
    
    FZ_trim = trim_fat(FZ)
    #print (FZ_trim)
    #print (FZ_in_HAZ)
    FZ_array = np.append(FZ_trim, FZ_in_HAZ)
    
    return [HAZ_array, FZ_array]

#assuming the material is 740H, the filename should have all the meta data
def meta_740H(filename):
    mlist = re.split("( )",filename)
    material = mlist[0]
    temperature = int(mlist[2])
    strain = int(mlist[6])
    FZ_number = int(mlist[18])
    time= int(mlist[26])
    time_type = (re.split("\\.",mlist[-1]))[0]
    
    #print (mlist)
    #print (material, temperature, strain, FZ_number, time, time_type)
    
    return [material, temperature, strain, FZ_number, time, time_type]
    
test_file= '740H 700 c 0 pct 0 v1 is fz 4 down into fz 100 hr.xlsx'
meta_data = meta_740H(test_file)
#df = pd.read_excel (test_file) #for an earlier version of Excel, you may need to use the file extension of 'xls' print (df)

#HAZ_array = trim_fat (df)

#need to make meta-data that identifies the number of indents in the FZ
#HAZ_indents_in_FZ = 6
#[HAZ_array, FZ_append] = poprow(HAZ_array, HAZ_indents_in_FZ)

[HAZ_array, FZ_array] = get_hard_740H(test_file, meta_data[3])

print (HAZ_array)

[416 387 431 382 382 393 418 423 411 416 426 418 400 426 416 426 402 436
 418 428 443 436 421 414 416 436 421 421 402 418 414 426 421 423 438 428
 433 414 418 400 414 421 414 414 409 400 411 426 404 426 414 411 411 421
 421 426 423 436 431 409 409 438 418 443 414 428 423 428 421 454 436 418
 418 431 426 433 433 409 414 407 421 414 425 416 416 393 411 426 407 426
 441 423 426 416 423 409 414 426 426 407 414 423 421 425 423 402 402 391
 418 405 405 449 421 411 411 438 426 416 421 389 414 426 411 409 409 428
 409 411 404 389 418 414 402 414 387 398 407 411 402 402 400 404 418 407
 416 423 416 407 402 387 402 411 387 395 398 393 436 405 418 421 400 421
 409 414 418 423 398 418 405 385 400 407 398 402 391 393 405 405 411 404
 418 418 396 396 418 396 398 428 400 398 409 404 404 407 396 391 409 411
 404 411 421 402 414 387 407 404 414 391 400 398 402 393]


In [80]:
import pandas as pd

??pd.ExcelFile

In [81]:
import pandas as pd

test_file= '740H 700 c 0 pct 0 v1 is fz 4 down into fz 100 hr.xlsx'

xlsx = pd.ExcelFile(test_file)
hardness_sheets = []
for sheet in xlsx.sheet_names:
    hardness_sheets.append(xlsx.parse(sheet))
    
vector_number = len(hardness_sheets)

for vector in range(vector_number):
    hardness_trace = hardness_sheets[vector]
    